In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from elasticsearch import Elasticsearch

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client =  Groq(api_key = os.environ['GROQ_API_KEY'])


In [3]:
with open('../data/vietnamese_rag/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/home/codespace/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
len(model.encode("This is a simple sentence"))

384

In [6]:
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '220d97b49b71', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Bjagydp1SHuMN5WDZF6iog', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "context": {"type": "text"},
            "question": {"type": "text"},
            "answer": {"type": "text"}, 
            "group": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}

In [8]:
index_name = "vietnamese-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vietnamese-questions'})

In [9]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector1.pkl', 'rb') as file:
    operations1 = pickle.load(file)

In [10]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector2.pkl', 'rb') as file:
    operations2 = pickle.load(file)

In [11]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector3.pkl', 'rb') as file:
    operations3 = pickle.load(file)

In [12]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector4.pkl', 'rb') as file:
    operations4 = pickle.load(file)

In [13]:
with open('../data/vietnamese_rag/text_vector_pickle/text_vector5.pkl', 'rb') as file:
    operations5 = pickle.load(file)

In [14]:
for doc in tqdm(operations1):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:25<00:00, 46.89it/s]


In [15]:
for doc in tqdm(operations2):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:25<00:00, 47.04it/s]


In [16]:
for doc in tqdm(operations3):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1217/1217 [00:26<00:00, 45.23it/s]


In [17]:
for doc in tqdm(operations4):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 2028/2028 [00:43<00:00, 46.23it/s]


In [18]:
for doc in tqdm(operations5):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|█████████████████████| 1221/1221 [00:26<00:00, 46.27it/s]


In [19]:
search_term = "Theo bài viết, chủ nghĩa thân hữu gây ra những vấn đề gì cho xã hội?"
vector_search_term = model.encode(search_term)

In [20]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [21]:
res = es_client.search(index=index_name, knn=query, source=[ "answer", "question", "group"])
res["hits"]["hits"]

[{'_index': 'vietnamese-questions',
  '_id': 'sbRLLJEBFdjf_YyEEYhw',
  '_score': 0.8420652,
  '_source': {'question': 'Dân tộc của chúng ta có ảnh hưởng đến cơ hội sống của chúng ta không?',
   'answer': 'Vâng, dân tộc của chúng tôi có ảnh hưởng đến cơ hội sống của chúng tôi. Điều này được thấy ở nhiều khía cạnh khác nhau của cuộc sống bao gồm nền tảng tầng lớp xã hội, vai trò giới [1] [2], trình độ học vấn [3] [4] và thậm chí cả kết quả sức khỏe liên quan đến xu hướng tình dục [5]. Ví dụ, dân tộc được cho là có tương quan với sự chênh lệch về trình độ học vấn giữa các nhóm dân tộc thiểu số khác nhau, cho thấy rằng một số dân tộc đạt được thành tích thấp hơn những dân tộc khác [4]. Cũng có bằng chứng cho thấy vai trò của các thể chế xã hội có thể không phải lúc nào cũng tích cực và đôi khi có thể có lợi cho các cơ hội có quyền lực, có thể tác động đến các cá nhân thuộc các nhóm dân tộc khác nhau [1] [2].',
   'group': 'Expert'}},
 {'_index': 'vietnamese-questions',
  '_id': 'qLRKLJEBFd

In [22]:

import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['group']}-{doc['question']}-{doc['context']}-{doc['answer']}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [23]:

for doc in documents:
    doc['id'] = generate_document_id(doc)

In [77]:
documents[0]

{'group': 'General',
 'context': 'ngữ cảnh 1: Vượt qua nỗi sợ hãi, Minh Tú xuất sắc vào chung kết Asia\'s Next Top Model.  Bỏ qua hết những lo lắng về căn bệnh tim, Minh Tú đã hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng và dành chiếc vé vào đêm chung kết Asia’s Next Top Model..  Tối ngày 21/6, tập 12 của chương trình Asia’ Next Top Model mùa thứ 5 đã được lên sóng. 4 thí sinh cùng giám khảo bí mật Xiao Qing bước vào phần thi thử thách đặc biệt là đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng. Các cô gái phải vừa tạo dáng vừa thể hiện được sự mạnh mẽ, quyến rũ riêng.. Mặc dù có bệnh về tim nhưng Minh Tú đã cố gắng hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách này. Thí sinh chiến thắng là Shikin và đại diện Việt Nam cũng không kém cạnh gì khi xuất sắc xếp ở vị trí thứ 2, báo GĐVN đưa tin.. Bước vào thử thách, Minh Tú hét lớn: “Tôi còn nhiều việc chưa làm được đâu” vì sợ sẽ rơi xuống tuy nhiên 

In [80]:

with open('../data/vietnamese_rag/documents-with-ids1.json', 'wt') as f_out:
    json.dump(documents[:1217] , f_out, indent=2)

In [81]:
with open('../data/vietnamese_rag/documents-with-ids2.json', 'wt') as f_out:
    json.dump(documents[1217:1217*2] , f_out, indent=2)

In [82]:
with open('../data/vietnamese_rag/documents-with-ids3.json', 'wt') as f_out:
    json.dump(documents[1217*2:1217*3] , f_out, indent=2)

In [83]:
with open('../data/vietnamese_rag/documents-with-ids4.json', 'wt') as f_out:
    json.dump(documents[1217*3:1217*4] , f_out, indent=2)

In [84]:
with open('../data/vietnamese_rag/documents-with-ids5.json', 'wt') as f_out:
    json.dump(documents[1217*4] , f_out, indent=2)